In [9]:
import sys
import os
import random
from tqdm import tqdm
import pandas as pd
from nomic import embed
import tiktoken
from gpt4all import GPT4All
import time

In [10]:
import functions as fn

In [11]:
url = "https://raw.githubusercontent.com/chen700564/RGB/master/data/en.json"
data = fn.process_json(url)
data = random.sample(data, 1)
queries = [item["query"] for item in data]
answers = [item["answer"][0] for item in data]

In [12]:
prompt_template = """
You are an AI assistant specializing in Question Answering. Your task is to read the provided context carefully and then generate the most accurate and concise answer to the question based on the context.

Context: {context_concat}

Question: {query}

Answer:
"""

In [14]:
gen_model_1 = GPT4All("gpt4all-13b-snoozy-q4_0.gguf")
gen_model_1.name = "gpt4all-13b-snoozy-q4_0.gguf"

start_time = time.time()

models = [gen_model_1]
separator = " <|> "
max_total_tokens = 1600
enc = tiktoken.encoding_for_model("gpt-3.5-turbo")

contexts = {
    "positive": lambda positive, negative: separator.join(positive),
    "negative": lambda positive, negative: separator.join(negative),
    "posneg": lambda positive, negative: separator.join(positive) + separator + separator.join(negative)
}

for model in models:
    results = []
    for query, positive_context, negative_context, answer in tqdm(zip(queries, [item["positive"] for item in data], [item["negative"] for item in data], answers), total=len(queries)):

        result = {
            'Query': query,
            'Correct Answer': answer,
        }

        with model.chat_session():
            for context_type in contexts:
                context_concat = contexts[context_type](positive_context, negative_context)
                context_tokens = enc.encode(context_concat)
                limited_tokens = context_tokens[:max_total_tokens]
                context_concat_limited = enc.decode(limited_tokens)
                prompt = prompt_template.format(context_concat=context_concat_limited, query=query)
                prompt_tokens_lenght = len(enc.encode(prompt))
                # print(f'Prompt Number of Tokens: {prompt_tokens_lenght}')
                generated_answer = model.generate(prompt)
                result.update({f'{context_type.capitalize()} Predicted Answer': generated_answer})

        results.append(result)

    results_df = pd.DataFrame(results)

    results_df['EM Positive'] = results_df.apply(lambda row: fn.apply_exact_match(row, 'Positive Predicted Answer', 'Correct Answer'), axis=1)
    results_df['EM Negative'] = results_df.apply(lambda row: fn.apply_exact_match(row, 'Negative Predicted Answer', 'Correct Answer'), axis=1)
    results_df['EM PosNeg'] = results_df.apply(lambda row: fn.apply_exact_match(row, 'Posneg Predicted Answer', 'Correct Answer'), axis=1)

    results_df['Jaccard Positive'] = results_df.apply(lambda row: fn.apply_jaccard(row, 'Positive Predicted Answer', 'Correct Answer'), axis=1)
    results_df['Jaccard Negative'] = results_df.apply(lambda row: fn.apply_jaccard(row, 'Negative Predicted Answer', 'Correct Answer'), axis=1)
    results_df['Jaccard PosNeg'] = results_df.apply(lambda row: fn.apply_jaccard(row, 'Posneg Predicted Answer', 'Correct Answer'), axis=1)

    results_df['Cosine Positive'] = results_df.apply(lambda row: fn.apply_cosine(row, 'Positive Predicted Answer', 'Correct Answer'), axis=1)
    results_df['Cosine Negative'] = results_df.apply(lambda row: fn.apply_cosine(row, 'Negative Predicted Answer', 'Correct Answer'), axis=1)
    results_df['Cosine PosNeg'] = results_df.apply(lambda row: fn.apply_cosine(row, 'Posneg Predicted Answer', 'Correct Answer'), axis=1)

    # Save results
    filename = f"/results/{model.name}.json"
    results_df.to_json(filename, orient='records', lines=True)

end_time = time.time()
print(f"Total execution time: {(end_time - start_time) / 60} minutes.")

  0%|          | 0/1 [00:00<?, ?it/s]LLaMA: reached the end of the context window so resizing
LLaMA: reached the end of the context window so resizing
100%|██████████| 1/1 [15:52<00:00, 952.58s/it]


OSError: Cannot save file into a non-existent directory: '/results'

In [15]:
results_df

,Query,Correct Answer,Positive Predicted Answer,Negative Predicted Answer,Posneg Predicted Answer,EM Positive,EM Negative,EM PosNeg,Jaccard Positive,Jaccard Negative,Jaccard PosNeg,Cosine Positive,Cosine Negative,Cosine PosNeg
0,When did Apple M2 chip?,"[June 6, Jun 6, Jun. 6, 6 June, 6 Jun, 6 Jun.]","\nThe answer is ""June 6, 2022"".",The first models featuring the new M2 chip wer...,The first models featuring the new M2 chip wer...,0,0,0,0.5,0.090909,0.1,0.707877,0.303504,0.256748
